In [2]:
import os
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pandas.tseries.holiday import USFederalHolidayCalendar
from datetime import datetime, timedelta


In [20]:
#  Main Execution 
#First Get all the CSV files that have covid tweet realated information
folder_path = 'C:\\work\\W203\\COVID-19\\csse_covid_19_data\\csse_covid_19_daily_reports_us'
states = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming","District of Columbia"]

covid_df = pd.read_csv(folder_path + '\\04-02-2021.csv')
# update the datetime column to just DATE
covid_df['Last_Update'] = pd.to_datetime(covid_df['Last_Update']).dt.date
# remove puerto rico, grum and other non-state entities
covid_df = covid_df[covid_df.Province_State.isin(states)]

covid_df.head()
covid_df.shape


(51, 18)

In [21]:
# rename columns and keep only columns we need

covid_df = covid_df.rename(columns={"Confirmed":"total_covid_cases"})
covid_df = covid_df.rename(columns={"Deaths":"total_covid_deaths"})
covid_df = covid_df[['Province_State','FIPS','Last_Update','total_covid_cases','total_covid_deaths','Case_Fatality_Ratio','Total_Test_Results']]
covid_df.head()

,Province_State,FIPS,Last_Update,total_covid_cases,total_covid_deaths,Case_Fatality_Ratio,Total_Test_Results
0,Alabama,1.0,2021-04-03,516266,10607,2.054561,2411593.0
1,Alaska,2.0,2021-04-03,63501,313,0.492906,1911934.0
3,Arizona,4.0,2021-04-03,843132,16989,2.014987,8559009.0
4,Arkansas,5.0,2021-04-03,330753,5639,1.704898,2864503.0
5,California,6.0,2021-04-03,3675191,59614,1.622065,53934609.0


In [26]:

# add the new columns
covid_df['sum_facemask_enforced'] = 0
covid_df['sum_facemask_citation_fine'] = 0
covid_df['sum_holidays'] = 0
covid_df['sum_quarantine_travellers'] = 0
covid_df['sum_shelter_in_place'] = 0
#lower case all column names
covid_df.columns = map(str.lower, covid_df.columns)
#reset index
covid_df = covid_df.reset_index(drop=True)
covid_df.head()

,province_state,fips,last_update,total_covid_cases,total_covid_deaths,case_fatality_ratio,total_test_results,sum_facemask_enforced,sum_facemask_citation_fine,sum_holidays,sum_quarantine_travellers,sum_shelter_in_place
0,Alabama,1.0,2021-04-03,516266,10607,2.054561,2411593.0,0,0,0,0,0
1,Alaska,2.0,2021-04-03,63501,313,0.492906,1911934.0,0,0,0,0,0
2,Arizona,4.0,2021-04-03,843132,16989,2.014987,8559009.0,0,0,0,0,0
3,Arkansas,5.0,2021-04-03,330753,5639,1.704898,2864503.0,0,0,0,0,0
4,California,6.0,2021-04-03,3675191,59614,1.622065,53934609.0,0,0,0,0,0


In [28]:
# Get holidays between 3/2020 and 4/2021

cal = USFederalHolidayCalendar()
holidays = cal.holidays(start='3/1/2020', end='4/3/2021').to_pydatetime()
holidays


array([datetime.datetime(2020, 5, 25, 0, 0),
       datetime.datetime(2020, 7, 3, 0, 0),
       datetime.datetime(2020, 9, 7, 0, 0),
       datetime.datetime(2020, 10, 12, 0, 0),
       datetime.datetime(2020, 11, 11, 0, 0),
       datetime.datetime(2020, 11, 26, 0, 0),
       datetime.datetime(2020, 12, 25, 0, 0),
       datetime.datetime(2021, 1, 1, 0, 0),
       datetime.datetime(2021, 1, 18, 0, 0),
       datetime.datetime(2021, 2, 15, 0, 0)], dtype=object)

In [30]:
num_holidays = len(holidays)
covid_df['sum_holidays'] = num_holidays
covid_df.head()

,province_state,fips,last_update,total_covid_cases,total_covid_deaths,case_fatality_ratio,total_test_results,sum_facemask_enforced,sum_facemask_citation_fine,sum_holidays,sum_quarantine_travellers,sum_shelter_in_place
0,Alabama,1.0,2021-04-03,516266,10607,2.054561,2411593.0,0,0,10,0,0
1,Alaska,2.0,2021-04-03,63501,313,0.492906,1911934.0,0,0,10,0,0
2,Arizona,4.0,2021-04-03,843132,16989,2.014987,8559009.0,0,0,10,0,0
3,Arkansas,5.0,2021-04-03,330753,5639,1.704898,2864503.0,0,0,10,0,0
4,California,6.0,2021-04-03,3675191,59614,1.622065,53934609.0,0,0,10,0,0


In [31]:
#get data from csv that has info on mask mandates and other items we need

csv_mask_df = pd.read_csv(os.getcwd() + '\\covid_face_mask_mandate.csv')
csv_mask_df

,State,Public face mask mandate,Second mandate for facemasks by all individuals in public places,Face mask mandate enforced by fines,Face mask mandate enforced by criminal charge/citation,No legal enforcement of face mask mandate,Business face mask mandate,End face mask mandate,Attempt by state government to prevent local governments from implementing face mask orders,Stay at home/shelter in place,Religious Gatherings Exempt Without Clear Social Distance Mandate*,Stay-at-home order issued but did not specifically restrict movement of the general public,End stay at home/shelter in place,Quarantine mandate for some travelers,Quarantine mandate ended
0,Alabama,7/16/2020,0,0,0,1,5/11/2020,0,0,4/4/2020,0,0,4/30/2020,0,0
1,Alaska,4/24/2020,0,1,1,0,4/24/2020,5/22/2020,0,3/28/2020,0,0,4/24/2020,3/25/2020,2/14/2021
2,Arizona,0,0,0,0,0,5/8/2020,0,3/30/2020,3/31/2020,1,0,5/16/2020,4/9/2020,5/12/2020
3,Arkansas,7/20/2020,0,1,1,0,5/11/2020,0,0,0,0,0,0,5/5/2020,6/15/2020
4,California,6/18/2020,0,0,0,1,5/5/2020,0,0,3/19/2020,0,0,1/25/2021,0,0
5,Colorado,7/16/2020,0,0,0,0,4/23/2020,0,0,3/26/2020,1,0,4/27/2020,0,0
6,Connecticut,4/20/2020,0,0,0,1,4/3/2020,0,0,0,0,3/23/2020,5/20/2020,6/25/2020,3/19/2021
7,Delaware,4/28/2020,0,1,1,0,5/1/2020,0,0,3/24/2020,1,0,6/1/2020,3/30/2020,6/1/2020
8,District of Columbia,4/17/2020,0,1,0,0,4/15/2020,0,0,4/1/2020,0,0,5/29/2020,7/27/2020,0
9,Florida,0,0,0,0,0,5/11/2020,0,0,4/3/2020,1,0,5/18/2020,3/24/2020,8/5/2020


In [47]:
# loop thru ever state and calcualte values

for index, row in csv_mask_df.iterrows():
    state = row['State']
    public_facemask_mandate = row['Public face mask mandate']
    facemask_enforced_by_fines= str(row['Face mask mandate enforced by fines'])
    facemask_enforced_citation= str(row['Face mask mandate enforced by criminal charge/citation'])
    attempt_to_prevent_local_mask_enforcement= row['Attempt by state government to prevent local governments from implementing face mask orders']
    end_face_mask_mandate = str(row['End face mask mandate'])
    stay_at_home_start = str(row['Stay at home/shelter in place'])    
    stay_at_home_end = str(row['End stay at home/shelter in place']) 
    quarantine_start = str(row['Quarantine mandate for some travelers']) 
    quarantine_end = str(row['Quarantine mandate ended']) 
   
    
    dt_public_facemask_mandate = 0
    dt_end_face_mask_mandate = 0
    dt_stay_at_home_start = 0
    dt_stay_at_home_end = 0
    dt_quarantine_start = 0
    dt_quarantine_end = 0
    
    if '/' in public_facemask_mandate:
        dt_public_facemask_mandate = datetime.strptime(public_facemask_mandate,'%m/%d/%Y')
        
    if '/' in end_face_mask_mandate:
        dt_end_face_mask_mandate = datetime.strptime(end_face_mask_mandate,'%m/%d/%Y')
            
    if '/' in stay_at_home_start:
        dt_stay_at_home_start = datetime.strptime(stay_at_home_start,'%m/%d/%Y')
        
    if '/' in stay_at_home_end:
        dt_stay_at_home_end = datetime.strptime(stay_at_home_end,'%m/%d/%Y')
        
    if '/' in quarantine_start:
        dt_quarantine_start = datetime.strptime(quarantine_start,'%m/%d/%Y')
        
    if '/' in quarantine_end:
        dt_quarantine_end = datetime.strptime(quarantine_end,'%m/%d/%Y')

        
#covid_df['sum_facemask_enforced'] = 0
#covid_df['sum_facemask_citation_fine'] = 0
#covid_df['sum_holidays'] = 0
#covid_df['sum_quarantine_travellers'] = 0
#covid_df['sum_shelter_in_place'] = 0        

# todo : use udpate_where in pandas instead of looping over whole DF

    for index, row in covid_df.iterrows():
        if row['province_state'] == state:
            print(state)
            dt_delta_days = 0
            if dt_public_facemask_mandate != 0 and dt_end_face_mask_mandate != 0:
                dt_delta = dt_end_face_mask_mandate - dt_public_facemask_mandate
                print("Delta", dt_delta.days)
                dt_delta_days = dt_delta.days
                covid_df.at[index,'sum_facemask_enforced'] = dt_delta_days
            elif dt_public_facemask_mandate != 0:
                print("no end date")
                dt_delta = datetime.strptime('4/2/2021','%m/%d/%Y') - dt_public_facemask_mandate
                print("Delta", dt_delta.days)
                dt_delta_days = dt_delta.days
                covid_df.at[index,'sum_facemask_enforced'] = dt_delta_days
   
            if '1' in facemask_enforced_by_fines or '1' in  facemask_enforced_citation:
                covid_df.at[index,'sum_facemask_citation_fine'] = dt_delta_days
    

            if dt_stay_at_home_start != 0 and dt_stay_at_home_end != 0:
                dt_delta = dt_stay_at_home_end - dt_stay_at_home_start
                print("Delta", dt_delta.days)
                dt_delta_days = dt_delta.days
                covid_df.at[index,'sum_shelter_in_place'] = dt_delta_days
            elif dt_stay_at_home_start != 0:
                print("no end date")
                dt_delta = datetime.strptime('4/2/2021','%m/%d/%Y') - dt_stay_at_home_start
                print("Delta", dt_delta.days)
                dt_delta_days = dt_delta.days
                covid_df.at[index,'sum_shelter_in_place'] = dt_delta_days
                
               
            if dt_quarantine_start != 0 and dt_quarantine_end != 0:
                dt_delta = dt_quarantine_end - dt_quarantine_start
                print("Delta", dt_delta.days)
                dt_delta_days = dt_delta.days
                covid_df.at[index,'sum_quarantine_travellers'] = dt_delta_days
            elif dt_quarantine_start != 0:
                print("no end date")
                dt_delta = datetime.strptime('4/2/2021','%m/%d/%Y') - dt_quarantine_start
                print("Delta", dt_delta.days)
                dt_delta_days = dt_delta.days
                covid_df.at[index,'sum_quarantine_travellers'] = dt_delta_days
                
      
    
covid_df
                


Alabama
no end date
Delta 260
Delta 26
Alaska
Delta 28
Delta 27
Delta 326
Arizona
Delta 46
Delta 33
Arkansas
no end date
Delta 256
Delta 41
California
no end date
Delta 288
Delta 312
Colorado
no end date
Delta 260
Delta 32
Connecticut
no end date
Delta 347
Delta 267
Delaware
no end date
Delta 339
Delta 69
Delta 63
District of Columbia
no end date
Delta 350
Delta 58
no end date
Delta 249
Florida
Delta 45
Delta 134
Georgia
Delta 28
Hawaii
Delta 67
no end date
Delta 372
Idaho
Delta 37
Delta 31
Illinois
no end date
Delta 336
Delta 69
Indiana
no end date
Delta 249
Delta 54
Iowa
Delta 83
Kansas
no end date
Delta 273
Delta 35
no end date
Delta 375
Kentucky
no end date
Delta 326
Delta 34
Louisiana
no end date
Delta 238
Delta 53
Maine
no end date
Delta 336
Delta 59
no end date
Delta 364
Maryland
no end date
Delta 349
Delta 46
no end date
Delta 106
Massachusetts
no end date
Delta 331
Delta 55
Delta 360
Michigan
no end date
Delta 340
Delta 69
Minnesota
no end date
Delta 252
Delta 51
Mississippi
D

,province_state,fips,last_update,total_covid_cases,total_covid_deaths,case_fatality_ratio,total_test_results,sum_facemask_enforced,sum_facemask_citation_fine,sum_holidays,sum_quarantine_travellers,sum_shelter_in_place
0,Alabama,1.0,2021-04-03,516266,10607,2.054561,2411593.0,260,0,10,0,26
1,Alaska,2.0,2021-04-03,63501,313,0.492906,1911934.0,28,28,10,326,27
2,Arizona,4.0,2021-04-03,843132,16989,2.014987,8559009.0,0,0,10,33,46
3,Arkansas,5.0,2021-04-03,330753,5639,1.704898,2864503.0,256,256,10,41,41
4,California,6.0,2021-04-03,3675191,59614,1.622065,53934609.0,288,0,10,0,312
5,Colorado,8.0,2021-04-03,465330,6118,1.314766,7002984.0,260,0,10,0,32
6,Connecticut,9.0,2021-04-03,313956,7904,2.517550,7232655.0,347,0,10,267,267
7,Delaware,10.0,2021-04-03,95590,1565,1.637201,1576023.0,339,339,10,63,69
8,District of Columbia,11.0,2021-04-03,44807,1069,2.385788,1379522.0,350,350,10,249,58
9,Florida,12.0,2021-04-03,2071015,33586,1.621717,20918247.0,0,0,10,134,45


In [48]:
#write to csv file
covid_df.to_csv('covid_states_mask_v2.csv') 
